<a href="https://colab.research.google.com/github/RedDawe/SMS_spam_classification/blob/master/LSTM_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
import pandas as pd
import numpy as np
import tensorflow as tf
import nltk

In [24]:
assert tf.executing_eagerly()

In [28]:
data = pd.read_csv('spam.csv', encoding='latin-1').drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis=1)

In [29]:
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [30]:
data = data.replace(to_replace='\d+', value='__number__', regex=True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in __number__ a wkly comp to win FA...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [74]:
#nltk.download('wordnet')
#lemmatizer = nltk.stem.PorterStemmer()
lemmatizer = nltk.stem.WordNetLemmatizer()

data['v2'] = data['v2'].apply(lemmatize_wrapper)
data.head()

Streaming output truncated to the last 5000 lines.
you? you?
['I', 'shall', 'book', 'chez', 'jules', 'for', 'half', 'eight,', 'if', "that's", 'ok', 'with', 'you?'] ['I', 'shall', 'book', 'chez', 'jules', 'for', 'half', 'eight,', 'if', "that's", 'ok', 'with', 'you?']
Hhahhaahahah Hhahhaahahah
rofl rofl
wtf wtf
nig nig
wa wa
leonardo leonardo
in in
your your
room room
or or
something something
['Hhahhaahahah', 'rofl', 'wtf', 'nig', 'wa', 'leonardo', 'in', 'your', 'room', 'or', 'something'] ['Hhahhaahahah', 'rofl', 'wtf', 'nig', 'wa', 'leonardo', 'in', 'your', 'room', 'or', 'something']
Yep, Yep,
at at
derek's derek's
house house
now, now,
see see
you you
Sunday Sunday
&lt;__number__ &lt;__number__
['Yep,', 'at', "derek's", 'house', 'now,', 'see', 'you', 'Sunday', '&lt;__number__'] ['Yep,', 'at', "derek's", 'house', 'now,', 'see', 'you', 'Sunday', '&lt;__number__']
It's It's
cool, cool,
let let
me me
know know
before before
it it
kick kick
off off
around around
&lt;#&gt; &lt;#&gt;
, ,
I'l

KeyboardInterrupt: ignored

In [80]:
lemmatizer.lemmatize('eggs')

'egg'

In [93]:
def lemmatize_wrapper(text):
  words = text.split()
  lemmed = []
  for word in words:
    lemmed.append(lemmatizer.lemmatize(word))
    print(lemmatizer.lemmatize(word), word, word=='eggs', [char for char in word])

  print(lemmed, words)
  return ' '.join(lemmed)

In [94]:
lemmatize_wrapper('Garbage bags, eggs, jam, bread, hannaford wheat chex')

Garbage Garbage False ['G', 'a', 'r', 'b', 'a', 'g', 'e']
bags, bags, False ['b', 'a', 'g', 's', ',']
eggs, eggs, False ['e', 'g', 'g', 's', ',']
jam, jam, False ['j', 'a', 'm', ',']
bread, bread, False ['b', 'r', 'e', 'a', 'd', ',']
hannaford hannaford False ['h', 'a', 'n', 'n', 'a', 'f', 'o', 'r', 'd']
wheat wheat False ['w', 'h', 'e', 'a', 't']
chex chex False ['c', 'h', 'e', 'x']
['Garbage', 'bags,', 'eggs,', 'jam,', 'bread,', 'hannaford', 'wheat', 'chex'] ['Garbage', 'bags,', 'eggs,', 'jam,', 'bread,', 'hannaford', 'wheat', 'chex']


'Garbage bags, eggs, jam, bread, hannaford wheat chex'

In [31]:
X = data['v2']
Y = data['v1']

Y.where(Y == 'ham', 1, inplace=True)
Y.where(Y == 1, 0, inplace= True)

Y.head()

0    0
1    0
2    1
3    0
4    0
Name: v1, dtype: object

In [32]:
Y_test = np.array(Y[5000:], dtype=np.int32)
Y_cv = np.array(Y[4000:5000], dtype=np.int32)
Y_train = np.array(Y[:4000], dtype=np.int32)

X_test = X[5000:]
X_cv = X[4000:5000]
X_train = X[:4000]

In [33]:
spam_perc = np.sum(Y_train)/Y_train.shape[0]*100
print(spam_perc)

weight_dict = {
    0: 1,
    1: 2
}

13.375


In [34]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X_train)

def preprocess(untokenized):
  sequences = tokenizer.texts_to_sequences(untokenized)
  sequence_matrix = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=150)

  return sequence_matrix

In [35]:
def recall_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        possible_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + tf.keras.backend.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_true * y_pred, 0, 1)))
        predicted_positives = tf.keras.backend.sum(tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + tf.keras.backend.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+tf.keras.backend.epsilon()))

In [41]:
inputs = tf.keras.Input(shape=[150])
model = tf.keras.layers.Embedding(1000, 100, input_length=150)(inputs)
model = tf.keras.layers.LSTM(64, activation='tanh', return_sequences=True)(model)
model = tf.keras.layers.LSTM(1, activation='sigmoid')(model)
model = tf.keras.Model(inputs=inputs, outputs=model)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', f1_m, tf.keras.metrics.AUC()])
model.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 150)]             0         
_________________________________________________________________
embedding_3 (Embedding)      (None, 150, 100)          100000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 150, 64)           42240     
_________________________________________________________________
lstm_7 (LSTM)                (None, 1)                 264       
Total params: 142,504
Trainable params: 142,504
Non-trainable params: 0
_________________________________________________________________


In [42]:
model.fit(preprocess(X_train), Y_train, batch_size=128, epochs=10, class_weight=weight_dict)

Epoch 1/10
32/32 [==============================] - 11s 340ms/step - loss: 0.4781 - accuracy: 0.8838 - f1_m: 0.1763 - auc: 0.8610
Epoch 2/10
32/32 [==============================] - 11s 334ms/step - loss: 0.1472 - accuracy: 0.9775 - f1_m: 0.9147 - auc: 0.9880
Epoch 3/10
32/32 [==============================] - 11s 338ms/step - loss: 0.0650 - accuracy: 0.9877 - f1_m: 0.9552 - auc: 0.9968
Epoch 4/10
32/32 [==============================] - 11s 347ms/step - loss: 0.0445 - accuracy: 0.9908 - f1_m: 0.9669 - auc: 0.9978
Epoch 5/10
32/32 [==============================] - 11s 343ms/step - loss: 0.0299 - accuracy: 0.9950 - f1_m: 0.9797 - auc: 0.9990
Epoch 6/10
32/32 [==============================] - 11s 352ms/step - loss: 0.0243 - accuracy: 0.9965 - f1_m: 0.9883 - auc: 0.9985
Epoch 7/10
32/32 [==============================] - 11s 350ms/step - loss: 0.0157 - accuracy: 0.9977 - f1_m: 0.9923 - auc: 0.9996
Epoch 8/10
32/32 [==============================] - 11s 343ms/step - loss: 0.0118 - accura

In [43]:
model.evaluate(preprocess(X_cv), Y_cv)

32/32 [==============================] - 1s 31ms/step - loss: 0.1130 - accuracy: 0.9630 - f1_m: 0.8087 - auc: 0.9854


[0.11302634328603745,
 0.9629999995231628,
 0.8087391257286072,
 0.9854273200035095]

In [44]:
model.evaluate(preprocess(X_test), Y_test)

18/18 [==============================] - 1s 31ms/step - loss: 0.0906 - accuracy: 0.9755 - f1_m: 0.8187 - auc: 0.9903


[0.09056270122528076,
 0.9755244851112366,
 0.818749189376831,
 0.9903397560119629]